In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [13]:
SEASONS = list(range(2020,2024))

# pointer to the directories
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [14]:
async def get_html(url, selector, sleep = 1, retries = 3):
    html = None
    
    # Start trying
    for i in range(1,retries):
        time.sleep(sleep * i)
        
        try:
            async with async_playwright() as p:
                brower = await p.chromium.launch() # open brower
                page = await brower.new_page() # open a new page
                await page.goto(url) # go to the page
                print(await page.title() + url.split("/")[-1])
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
                print(f"Timeout error on {url}")
                continue
        else:
            break
        
    return html

In [15]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]
    
    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [16]:
for season in SEASONS:
    await scrape_season(season)

2019-20 NBA Schedule | Basketball-Reference.comNBA_2020_games.html
2019-20 NBA Schedule | Basketball-Reference.comNBA_2020_games-october.html
2019-20 NBA Schedule | Basketball-Reference.comNBA_2020_games-november.html
2019-20 NBA Schedule | Basketball-Reference.comNBA_2020_games-december.html
2019-20 NBA Schedule | Basketball-Reference.comNBA_2020_games-january.html
2019-20 NBA Schedule | Basketball-Reference.comNBA_2020_games-february.html
2019-20 NBA Schedule | Basketball-Reference.comNBA_2020_games-march.html
2019-20 NBA Schedule | Basketball-Reference.comNBA_2020_games-july.html
2019-20 NBA Schedule | Basketball-Reference.comNBA_2020_games-august.html
2019-20 NBA Schedule | Basketball-Reference.comNBA_2020_games-september.html
2020-21 NBA Schedule | Basketball-Reference.comNBA_2021_games.html
2021-22 NBA Schedule | Basketball-Reference.comNBA_2022_games.html
2022-23 NBA Schedule | Basketball-Reference.comNBA_2023_games.html


In [17]:
standings_files = os.listdir(STANDINGS_DIR)

In [18]:
async def scrape_game(standings_file):
    with open(standings_file, "r") as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [19]:
standing_files = [s for s in standings_files if ".html" in s]

In [23]:
standing_files.sort()
standing_files

['NBA_2020_games-august.html',
 'NBA_2020_games-december.html',
 'NBA_2020_games-february.html',
 'NBA_2020_games-january.html',
 'NBA_2020_games-july.html',
 'NBA_2020_games-march.html',
 'NBA_2020_games-november.html',
 'NBA_2020_games-october.html',
 'NBA_2020_games-september.html',
 'NBA_2021_games-april.html',
 'NBA_2021_games-december.html',
 'NBA_2021_games-february.html',
 'NBA_2021_games-january.html',
 'NBA_2021_games-july.html',
 'NBA_2021_games-june.html',
 'NBA_2021_games-march.html',
 'NBA_2021_games-may.html',
 'NBA_2022_games-april.html',
 'NBA_2022_games-december.html',
 'NBA_2022_games-february.html',
 'NBA_2022_games-january.html',
 'NBA_2022_games-june.html',
 'NBA_2022_games-march.html',
 'NBA_2022_games-may.html',
 'NBA_2022_games-november.html',
 'NBA_2022_games-october.html',
 'NBA_2023_games-april.html',
 'NBA_2023_games-december.html',
 'NBA_2023_games-february.html',
 'NBA_2023_games-january.html',
 'NBA_2023_games-june.html',
 'NBA_2023_games-march.html',
 '

In [28]:
for i in range(len(standing_files)):
    print(f"\nNow scrapping file: {standing_files[i]}\nProgress:{i}/{len(standing_files)}\n")
    filepath = os.path.join(STANDINGS_DIR, standing_files[i])
    await scrape_game(filepath)


Now scrapping file: NBA_2020_games-august.html
Progress:0/35


Now scrapping file: NBA_2020_games-december.html
Progress:1/35


Now scrapping file: NBA_2020_games-february.html
Progress:2/35


Now scrapping file: NBA_2020_games-january.html
Progress:3/35


Now scrapping file: NBA_2020_games-july.html
Progress:4/35


Now scrapping file: NBA_2020_games-march.html
Progress:5/35


Now scrapping file: NBA_2020_games-november.html
Progress:6/35


Now scrapping file: NBA_2020_games-october.html
Progress:7/35


Now scrapping file: NBA_2020_games-september.html
Progress:8/35


Now scrapping file: NBA_2021_games-april.html
Progress:9/35


Now scrapping file: NBA_2021_games-december.html
Progress:10/35


Now scrapping file: NBA_2021_games-february.html
Progress:11/35


Now scrapping file: NBA_2021_games-january.html
Progress:12/35


Now scrapping file: NBA_2021_games-july.html
Progress:13/35


Now scrapping file: NBA_2021_games-june.html
Progress:14/35


Now scrapping file: NBA_2021_games-march.

In [ ]:
lst = os.listdir(SCORES_DIR) # your directory path
number_files = len(lst)
print(number_files)

917
